# EC Onnx Inference

# Setting Up Necessary Things

In [1]:
# Jupyter Notebook Magic Command - Auto Reloading
%reload_ext autoreload
%autoreload 2

# Jupyter Notebook Magic Command - Inline Plotting
%matplotlib inline

In [2]:
# Ignore All Warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Check GPU Status
!nvidia-smi

Tue Sep 12 15:12:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/All Entertainment Classification

/content/drive/MyDrive/All Entertainment Classification


# Necessary Imports

In [6]:
! pip install -q transformers[sentencepiece] fastai ohmeow-blurr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
! pip install -q onnxruntime onnx onnxruntime-gpu onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.0 MB/s eta 0:00:00


In [8]:
from tqdm.notebook import tqdm
import numpy as np
from sklearn import metrics

import torch

from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer

from fastai.text.all import *

from blurr.text.data.all import *
from blurr.text.modeling.all import *

import onnxruntime as rt
from onnxruntime.quantization import quantize_dynamic, QuantType

# Data Cleaning

In [9]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/All Entertainment Classification/data/processed/category/entertainment_category.csv")
df.head()

,description,category
0,"What could be worse than letting billions die? In the near future, to escape the crush and clutter of a packed and polluted Earth, the world's elite flock to Atopia, an enormous corporate-owned artificial island in the Pacific Ocean. It is there that Dr. Patricia Killiam rushes to perfect the ultimate in virtual reality: a program to save the ravaged Earth from mankind's insatiable appetite for natural resources. The Atopia Chronicles (Book 1 of the Atopia series) is the tale of mankind's dark slide across the apocalypse as humans and machines merge in a world teetering on the brink of eco...",book
1,"Jorge Luis Borges declared The Invention of Morel a masterpiece of plotting, comparable to The Turn of the Screw and Journey to the Center of the Earth. Set on a mysterious island, Bioy's novella is a story of suspense and exploration, as well as a wonderfully unlikely romance, in which every detail is at once crystal clear and deeply mysterious. Inspired by Bioy Casares's fascination with the movie star Louise Brooks, The Invention of Morel has gone on to live a secret life of its own. Greatly admired by Julio Cortázar, Gabriel García Márquez, and Octavio Paz, the novella helped to usher ...",book
2,"Thee devil shall roam the earth And plague and terror shall reign Upon the world of man Heaven shattered Destruction of angels shall commence From his mighty seat the great one shall be cast down Impaled and defiled, his throne upon the pyre Inflamed with the seeds of misery The immortal blood will be spilt And the dark underworld will be unleashed A feeble prayer to failing deities To the ancient evil nothing is sacred A once powerful ruler, now a helpless whelp Bowing down before the eternal demon Roots rising From the dank earth piercing through the Savior's skin upwards from the pit of...",music
3,"(Hook) Them niggas over there, never play fair You cud get hit wit that stick They do not care You a fruity nigga, you a care bear We running wit static We splitting yo cabbage We dug u a hole We lessen your baggage You luggage in the attic I put that on a Patek Am i bout this shit nigga Oui Oui Oui Oui Oui Ey! (Verse 1) Them niggas that hate They asking dem questions They looking so clueless And then they debate But those who the realest They movin immune They fresh like Colgate We smoke us a pack We plan an/and attack We never back down We tossing to/the die And thats for a fact Money wa...",music
4,"Jigsaw Puzzle: Beach Season – the journey to the seaside starts right now! 500 high-quality photos means 500 ideas for a summer vacation! Take a virtual tour of the resorts of your dreams: Hawaii, Bali, the Maldives, Seychelles, the Caribbean… Enjoy exotic photos of these little corners of paradise on our planet. Romantic views, golden beaches, unique nature and incredible local cuisine. Jigsaw Puzzle: Beach Season will tell you where to find the best places on our planet for a calm and relaxing vacation and where adventure-seekers can go for some active rest. See local flora and fauna, ex...",game


In [10]:
print("Shape of Data: ", df.shape)

Shape of Data:  (42535, 2)


## Data Split

In [11]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)

train_ids, valid_ids = splitter(df)

print("Train Split: ", len(train_ids))
print("Valid Split: ", len(valid_ids))

Train Split:  38282
Valid Split:  4253


In [12]:
valid_df = df.loc[valid_ids]

valid_df.head()

,description,category
6377,"A young actress tries to help her mother's romance by deception, and winds up in some hilarious escapades before learning her lesson.",movie
31380,"U.C. 0095.12.03 The Neo Zeon remnants known as the Sleeves conduct a performance test on a certain new type of mobile suit in a debris sector. Having gotten wind of this intel, two warships head for the debris sector. One ship is a Earth Federation Forces warship. The other ship is an Anaheim Electronics testing ship. On board the Federation Forces battleship, a newly constructed Unicorn Gundam Unit 3 awaited its time to sortie… (Source: Zeonic Scanlations)",anime
33812,"A timeless novel of a straitlaced village's awakening to joy and sensuality - every page offers a description of chocolate to melt in the mouths of chocoholics, francophiles, armchair gourmets, cookbook readers, and lovers of passion everywhere.Illuminating Peter Mayle's South of France with a touch of Laura Esquivel's magic realism, Chocolat is a timeless novel of a straitlaced village's awakening to joy and sensuality. In tiny Lansquenet, where nothing much has changed in a hundred years, beautiful newcomer Vianne Rocher and her exquisite chocolate shop arrive and instantly begin to play...",book
26181,"Hatsune-Jima was once well-known for its cherry trees, which would bloom no matter what time of year it was. Two years ago, the island's cherry trees lost this ability and all became ordinary cherry trees that bloom only in the spring. Jun'ichi Asakura is now a graduating senior at Kazami Academy, and is surrounded by many close friends, old and new alike. On a stormy summer day, a mysterious young girl, Aisia, arrives at Jun'ichi's door. She is looking for Sakura's grandmother to study magic, ultimately hoping to bring happiness to everyone. When she discovers that grandmother Yoshino had...",anime
41041,"Nikos Kazantzakis (1883-1957) was born in and lived in Greece most of his life. He was the author of poetry, plays, articles and novels, including The Last Temptation of Christ, Zorba the Greek and The Greek Passion.",book


# Fastai & Blurr Inference

In [13]:
model_path = "models/entertainment-category-stage-1.pkl"
learner_inf = load_learner(model_path)

In [14]:
learner_inf.blurr_predict("random placeholder")

[{'label': 'a',
  'score': 0.8001508116722107,
  'class_index': 0,
  'class_labels': ['anime', 'book', 'game', 'movie', 'music', 'tv show'],
  'probs': [0.8001508116722107,
   0.009580004028975964,
   0.16299669444561005,
   0.018679799512028694,
   0.007327580824494362,
   0.0012650785502046347]}]

In [15]:
learner_inf.blurr_predict("random placeholder")[0]["label"]

'a'

In [16]:
class_index = learner_inf.blurr_predict("random placeholder")[0]['class_index']

class_label = learner_inf.blurr_predict("random placeholder")[0]["class_labels"][class_index]

print(class_label)

anime


# Evaluation

In [35]:
def metric_measures(test_df, pred_df):
  targets = [target for target in test_df["category"].to_list()]
  outputs = [pred for pred in pred_df]

  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average="micro")

  print(f"Accuracy: {accuracy}")
  print(f"F1 Score (Micro): {f1_score_micro}")

In [36]:
preds = []

for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row["description"]
  class_index = learner_inf.blurr_predict(desc)[0]['class_index']
  class_label = learner_inf.blurr_predict(desc)[0]["class_labels"][class_index]
  preds.append(class_label)

  0%|          | 0/4253 [00:00<?, ?it/s]

In [37]:
metric_measures(valid_df, preds)

Accuracy: 0.9466259111215612
F1 Score (Micro): 0.9466259111215612


# Convert to ONNX

In [39]:
model_path = "models/entertainment-category-stage-1.pkl"
learner_inf = load_learner(model_path)

In [40]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    'models/entertainment-category.onnx',
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [41]:
onnx_model_path = 'models/entertainment-category.onnx'
quantized_onnx_model_path = 'models/entertainment-category-quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant

# ONNX Inference

## Normal ONNX

In [71]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = ['anime', 'book', 'game', 'movie', 'music', 'tv show']

inf_session = rt.InferenceSession('models/entertainment-category.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [69]:
preds = []

for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  mask = np.where(probs[0] == probs.max())[0][0]

  preds.append(class_labels[mask])

  0%|          | 0/4253 [00:00<?, ?it/s]

In [70]:
metric_measures(valid_df, preds)

Accuracy: 0.9482718081354338
F1 Score (Micro): 0.9482718081354338


## Quantized ONNX

In [72]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = ['anime', 'book', 'game', 'movie', 'music', 'tv show']

inf_session = rt.InferenceSession('models/entertainment-category-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [73]:
preds = []

for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  mask = np.where(probs[0] == probs.max())[0][0]

  preds.append(class_labels[mask])

  0%|          | 0/4253 [00:00<?, ?it/s]

In [74]:
metric_measures(valid_df, preds)

Accuracy: 0.9485069362802727
F1 Score (Micro): 0.9485069362802727
